Text Classification with a Bag-of-Words model
==============================================

This notebook demonstrates how we can train a classifier to predict a label for a text based on word counts.

The texts consist of 19th century novels in English from project Gutenberg. The selection is based on the "unsuccessful" and successful novels (low and high download counts) as used in Ashok et al (2013), [Success with style](http://aclweb.org/anthology/D13-1181).

We will consider the same classification task of successful vs unsuccessful novels.

First some preliminaries. Import the external libraries we will use.
Most importantly, we rely on Scikit-Learn to do machine learning. See http://scikit-learn.org/

In [2]:
import os
import numpy
import pandas
import matplotlib.pyplot as plt
import sklearn.feature_extraction
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.decomposition
import sklearn.linear_model
import sklearn.metrics

# Tweak how tables are displayed
pandas.set_option('display.precision', 4)
pandas.set_option('display.max_colwidth', 30)
pandas.set_option('display.colheader_justify', 'left')

Load the texts and convert to a bag-of-words table
--------------------------------------------------
The bag-of-words ([BOW](http://en.wikipedia.org/wiki/Bag-of-words_model)) table will contain a count for each text and every word.

All letters are converted to lower case and we filter the words until we only have words with two or more alphanumeric characters (no punctuation).

Important parameters are:

-  ``max_features`` limits the model to consider only the top *n* most frequent words. More is often better but may be slow to compute.
- ``min_df`` and ``max_df`` restrict the words to those that occur in a certain proportion of texts.

For example, setting these to the values 0.2 and 0.8, respectively, restricts the model to words that are in at least 20%, and at most 80% of the texts. This removes rare words on the one hand, and ignores highly frequent words such as [function words](http://en.wikipedia.org/wiki/Function_word) on the other.

- ``use_idf=True`` turns on [tf-idf](http://en.wikipedia.org/wiki/Tf%E2%80%93idf) weighting; this incorporates not only the total frequency of a word, but also the number of texts in which it appears. This means that words which are frequent in one text but not in others will get a high score, while words that are frequent in all documents receive a lower score.
- ``sublinear_tf=True`` is another variation which makes the word frequencies logarithmic.
- ``ngram_range``: selects the number of words in each feature; the default of ``(1, 1)`` selects individual words, which is most efficient. Higher numbers result in more informative features, but also requires a lot more memory. Bigrams, i.e., ``ngram_range=(2, 2)`` is a good trade-off and yields features with 2 consecutive words such as ``the man`` and ``man walks`` etc.

# transform() vs fit_transform()


fit means to fit the model to the data being provided. This is where the model "learns" from the data.

transform means to transform the data (produce model outputs) according to the fitted model.

fit_transform means to do both - Fit the model to the data, then transform the data according to the fitted model. Calling fit_transform is a convenience to avoid needing to call fit and transform sequentially on the same input.


In [13]:
# Set up a simple BOW model
# vectorizer is a variable that contains the instructions on how to read the text
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(
        input='filename', lowercase=True, token_pattern=r'\b[-\w]{1,}\b',
        min_df=0.2, max_df=0.8, max_features=10000,
        use_idf=True, sublinear_tf=False, ngram_range=(2, 3)) # two args min and max value

# max features refers to analyse the top 10000 ngrams. So it will count all the ngrams
# and give you the top 10000
# Get a list of all filenames in the 'train/' folder,
# and add their text to the BOW table 'X'.
histpd = pandas.read_csv("historical.csv")
histpd = pandas.read_csv('historical.csv')
filenames = list(histpd["Filename"])

#filenames = os.listdir('train/')

ft = []

for a in filenames:
    if a in os.listdir('train/'):
        ft.append(a)

# X = vectorizer.fit_transform(['train/' + a for a in filenames])
X = vectorizer.fit_transform(['train/' + a for a in ft])

# X contains the read text

# .fit_transform() Learn the vocabulary dictionary and return document-term matrix.
# vectorizer.fit_transform(...) collects the vocabulary of words/ngrams (the fit part),
# and converts the texts to a table of numbers (the transform part).
# the result of that is put in the variable X, which is just a 
# table of numbers, but the vectorizer knows which columns refer to which words/ngrams

[tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.

At initialization, vectorizer is empty. after you fit it on a corpus (line 10), yes, it has a list of cleaned ngrams and it knows how to convert a text to a vector of counts referring to those ngrams. When we initialise it we are just telling vectorizer how to read the text and what to store. 

the same thing happens later with the classifier. it is only trained when you use .fit()

In [14]:
print(X)

  (0, 660)	0.001618365771111227
  (0, 4018)	0.0014541083260844203
  (0, 3721)	0.0022084084026027114
  (0, 4511)	0.0014169248873963317
  (0, 5431)	0.0014541083260844203
  (0, 7033)	0.001574713760251304
  (0, 4988)	0.0013988372256013594
  (0, 2751)	0.0015963025078015506
  (0, 1091)	0.002946459258736117
  (0, 1656)	0.00205941875105873
  (0, 3452)	0.0017620158458614853
  (0, 7645)	0.0025612042607286944
  (0, 4222)	0.0014732296293680585
  (0, 6586)	0.0012966638821470084
  (0, 9373)	0.001618365771111227
  (0, 7054)	0.0015126008743788454
  (0, 9860)	0.0015126008743788454
  (0, 3278)	0.0015963025078015506
  (0, 9867)	0.0014927222432053588
  (0, 7466)	0.0023367324075895916
  (0, 5751)	0.0014353444513501817
  (0, 4554)	0.0017620158458614853
  (0, 3006)	0.0029854444864107176
  (0, 7330)	0.0014732296293680585
  (0, 9092)	0.0018423657630806207
  :	:
  (81, 5064)	0.0032532060309458377
  (81, 1181)	0.003084952763285674
  (81, 7151)	0.003084952763285674
  (81, 2789)	0.005167604757226258
  (81, 9099)	0

Inspect the vector representation
---------------------------------
To see what the vector representation looks like, we will look at how a simple [example](https://en.wikipedia.org/wiki/It_was_a_dark_and_stormy_night) is transformed to the BOW representation.

Notice that because of the parameters before, many words are ignored.

In [15]:
text = ('It was a dark and stormy night; '
        'the rain fell in torrents -- except at occasional intervals, '
        'when it was checked by a violent gust of wind which swept '
        'up the streets (for it is in London that our scene lies), '
        'rattling along the housetops, and fiercely agitating the scanty '
        'flame of the lamps that struggled against the darkness.')
with open('darkandstormy.txt', 'w', encoding='utf8') as out:
    out.write(text)
vec = vectorizer.transform(['darkandstormy.txt']) # Transform documents to document-term matrix. No training.

# we get back a large vector with a value for each possible word.
# show a table with only the non-zero items in the vector: 
feature_names = vectorizer.get_feature_names() # gets the actual words
pandas.DataFrame([(feature_names[b], (a, b), vec[a, b])
                  # from dictionary giving the word (value) , then using the (key) index
                        for a, b in zip(*vec.nonzero())], 
                       columns=['word', 'index', 'weight'])

,word,index,weight
0,when it was,"(0, 9404)",0.2590
1,the streets,"(0, 7958)",0.2757
2,the rain,"(0, 7805)",0.3015
3,that our,"(0, 7037)",0.2757
4,of wind,"(0, 5579)",0.3451
5,it is in,"(0, 4226)",0.3179
6,in london,"(0, 3954)",0.4066
7,for it is,"(0, 2351)",0.3095
8,fell in,"(0, 2244)",0.2590
9,a violent,"(0, 306)",0.3763


Non-zero items will exist for a document because it stores the words from previous documents it has read. 

The Index column refers to the index of the file and the index of the word, or more generally, the row and column in the array/matrix/table

[`zip()`](https://www.geeksforgeeks.org/zip-in-python/) maps the separate values into one tuple containing all aspects of the item.

Get label for each text
----------------------
The genre of each text, whether a text is a "success" or a "failure" (based on download counts), and some other information, is specified in a separate metadata file.

Note that in the original data, a text may have multiple genres, and an arbitrary genre was picked in this case. In a more careful study the single most appropriate genre would have to be selected by hand.

In [16]:
# Print the first 5 lines to see what the metadata looks like:
for line in list(open('historical.csv', encoding='utf8'))[:5]:
    print(line, end='')

Filename,Label,Genre,Fold,Success,Title,Author,Language,DownloadCount
11228.txt,Chesnutt_TheMarrowOfTradition,Historical,1,SUCCESS,the marrow of tradition,"chesnutt, charles w. (charles waddell), 1858-1932",en,147
11413.txt,Doyle_TheRefugeesATaleOfTwoContinents,Historical,1,SUCCESS,the refugees a tale of two continents,"doyle, arthur conan, sir, 1859-1930",en,129
17221.txt,Defoe_HistoryOfThePlagueInLondon,Historical,1,SUCCESS,history of the plague in london,"defoe, daniel, 1661?-1731",en,247
1880.txt,Cooper_PathfinderOrTheInlandSea,Historical,1,SUCCESS,"pathfinder; or, the inland sea","cooper, james fenimore, 1789-1851",en,185


the books are assigned to 5 "folds" randomly, to divide the corpus into train and test sets.

In [17]:
# Load the data; metadata.index will be the filename.
metadata = pandas.read_csv('historical.csv', index_col='Filename', encoding='utf8')
labels = dict(zip(metadata.index, metadata['Success']))

# collect the labels we want to predict
y = numpy.array([labels[a] for a in filenames])
print(y)
# Create an abbreviated label "Author_Title" for each text
authors = dict(zip(metadata.index, metadata['Author']))
titles = dict(zip(metadata.index, metadata['Title']))
abbrtitles = ['%s_%s' % (authors[a].split(',')[0].title(),
        titles[a][:15].title()) for a in filenames]

type(labels)

['SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS'
 'SUCCESS' 'SUCCESS' 'SUCCESS' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE'
 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'SUCCESS'
 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS'
 'SUCCESS' 'SUCCESS' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE'
 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'SUCCESS' 'SUCCESS'
 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS'
 'SUCCESS' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE'
 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'SUCCESS' 'SUCCESS' 'SUCCESS'
 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS'
 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE'
 'FAILURE' 'FAILURE' 'FAILURE' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS'
 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'SUCCESS' 'FAILURE'
 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAILURE'
 'FAIL

dict

Train a classifier
------------------

You can try different values for the parameter ``C``.
This parameter controls the level of regularization;
with higher values, the model will take more edge cases
(datapoints close to datapoints of other classes) into account.
This will give better scores on data that is similar to the training data,
but if the training data is not representative, it may result in more errors.

regularisation: ignore certain outliers. 

## Logistic Regression

[Logistic Regression Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression)in its basic form, models a binary dependent variable (win/lose, pass/fail). Models the probability of a certain event existing.
The function that converts log-odds to probability is the logistic function. 

The logistic regression model itself simply models probability of output in terms of input and does not perform statistical classification (which group does a new data point belong to?) (it is not a classifier), though it can be used to make a classifier, for instance by choosing a cutoff value and classifying inputs with probability greater than the cutoff as one class, below the cutoff as the other; this is a common way to make a binary classifier.

As opposed to a linear regression model which fits the data to a straight line, a logistic regression model fits the data to a logistic regression curve, which looks like an asymptote. |S|

Multinomial regression uses a dependent variable with more than two categories. 

## lbgfs

solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’

Algorithm to use in the optimization problem: 

- For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones.

- For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss; ‘liblinear’ is limited to one-versus-rest schemes.

- ‘newton-cg’, ‘lbfgs’, ‘sag’ and ‘saga’ handle L2 or no penalty

- ‘liblinear’ and ‘saga’ also handle L1 penalty

- ‘saga’ also supports ‘elasticnet’ penalty

[lbfgs](https://en.wikipedia.org/wiki/Limited-memory_BFGS): The algorithm's target problem is to minimize f(x) over unconstrained values (x can have any value) of the real-vector x where f is a differentiable scalar function (continuous). Trying to find the minimum value of the function. In this case, reducing the amount of error.

In [27]:
# Use the "Fold" column to split the training data into a train and test set.
# These folds were chosen in such a way that the labels are well balanced
# and all works by each other only occur in a single fold.
folds = dict(zip(metadata.index, metadata['Fold']))

# sss contains the split up sample
sss = sklearn.model_selection.PredefinedSplit([folds[a] for a in filenames])
# This returns train_index and test_index, which are arrays with indices of the datapoints
# that should be used for training and testing, respectively.


# Train a linear classifier and predict the genre of the items in the test set. 
# predict the genre?
# as before, giving instructions on how to read
clf = sklearn.linear_model.LogisticRegression(C=3, multi_class='multinomial', solver='lbfgs')

# X refers to the document-term matrix with the list of indices of cleaned ngrams
# X is a table of word counts, one document per row, one word/ngram per column.
# X[train_index] then selects only the word counts for the books that should be 
# trained on, without the books in test_index.
# y refers to the numpy array of title names of the texts in the corpus and whether they fail or succeed

# when we say sss.split(X,y) we are saying split X and y based on the instructions stored in sss
for train_index, test_index in sss.split(X, y): 
    clf.fit(X[train_index], y[train_index]) # fit => learning from the data then it predicts.
    pred = clf.predict(X[test_index]) 
    prob = clf.predict_proba(X[test_index])
print('Overall accuracy: %g %%' % sklearn.metrics.accuracy_score(y[test_index], pred))

IndexError: index (99) out of range

In [28]:
clf.classes_

array(['FAILURE', 'SUCCESS'], dtype='<U7')

Evaluate the classifier
-----------------------
The breakdown shows that not all genres are predicted as well;
the f-score column is the most important.

In the [confusion matrix](http://en.wikipedia.org/wiki/Confusion_matrix)
we can see which genres were mistaken most often.
The columns hold the number of times the model predicted a genre,
while the rows show the true genres.

specifically, the diagonal from the top left to the bottom right are the counts of correct predictions.

In [29]:
print(sklearn.metrics.classification_report(y[test_index], pred))
pandas.DataFrame(sklearn.metrics.confusion_matrix(y[test_index], pred),
                 index=clf.classes_, columns=clf.classes_)

ValueError: Found input variables with inconsistent numbers of samples: [20, 21]

Which words are most strongly associated with each genre?
---------------------------------------------------------
For each genre, the top 10 words most strongly linked to each genre are shown.

The words are ordered by the weight of the model for each genre combined with the average frequency of that word.

In [ ]:
# Sort the weights of the classifier and take top 10 items
topfeatures = {}
avgfreq = numpy.squeeze(numpy.asarray(X.mean(axis=0)))
if len(clf.classes_) > 2:
    for n, target in enumerate(clf.classes_):
        top10 = numpy.argsort(clf.coef_[n] * avgfreq)[::-1][:10]
        topfeatures[target] = pandas.DataFrame({
                'word': [feature_names[m] for m in top10],
                'score': (clf.coef_[n] * avgfreq)[top10]},
                index=range(1, 11))
else:
    # in case of a binary classification, negative weights are for the first class,
    # positive weights for the second
    top10 = numpy.argsort(clf.coef_[0] * avgfreq)[:10]
    topfeatures[clf.classes_[0]] = pandas.DataFrame({
            'word': [feature_names[m] for m in top10],
            'score': (clf.coef_[0] * avgfreq)[top10]},
            index=range(1, 11))
    top10 = numpy.argsort(clf.coef_[0] * avgfreq)[::-1][:10]
    topfeatures[clf.classes_[1]] = pandas.DataFrame({
            'word': [feature_names[m] for m in top10],
            'score': (clf.coef_[0] * avgfreq)[top10]},
            index=range(1, 11))
pandas.concat(topfeatures, axis=1)

Can the model predict labels for new texts?
---------------------------------------------

If you're done experimenting with the model parameters, we can load new texts that the model has never seen before, and see what it predicts.

In [ ]:
# Since we now evaluate on an external held-out set,
# we can use everything as training data
clf.fit(X, y)

# Transform the new files to the format of the existing BOW table
newfiles = os.listdir('test/')
X1 = vectorizer.transform(['test/' + a for a in newfiles])
y1 = numpy.array([labels[a] for a in newfiles])
pred = clf.predict(X1)
prob = clf.predict_proba(X1)

# Evaluate
print(sklearn.metrics.classification_report(y1, pred))
pandas.DataFrame(sklearn.metrics.confusion_matrix(y1, pred),
                       index=clf.classes_,
                       columns=clf.classes_)

Which books were the hardest to classify?
-----------------------------------------

The following table lists the Adventure books ordered by how confident the classifier is; i.e., ordered by the probability for the most likely label. A probability ranges from 0 to 1.

In the table after this the probabilities for each possible genre are given, and the probabilities of each row add up to 1.

In [ ]:
result = pandas.DataFrame([
            (authors[a].title(), titles[a].title(), p, labels[a], b)
        for a, b, p in zip(newfiles, pred, prob.max(axis=1))],
        index=newfiles,
        columns=['Author', 'Title', 'prob', 'actual', 'predicted'])
result.sort_values(by='prob')

In [ ]:
# Show probabilities of all labels for each novel (in case of 2 labels, this doesn't give extra information)
x = pandas.DataFrame(prob, index=newfiles, columns=clf.classes_)
x.loc[x.max(axis=1).sort_values().index, :]  # order from lowest to highest probability

Which words are most strongly associated with each label?
---------------------------------------------------------
For each label, the top 10 words most strongly linked to each label are shown.

The words are ordered by the weight of the model for each label combined with the average frequency of that word.

In [ ]:
# Sort the weights of the classifier and take top 10 items
topfeatures = {}
avgfreq = numpy.squeeze(numpy.asarray(X.mean(axis=0)))
if len(clf.classes_) > 2:
    for n, target in enumerate(clf.classes_):
        top10 = numpy.argsort(clf.coef_[n] * avgfreq)[::-1][:10]
        topfeatures[target] = pandas.DataFrame({
                'word': [feature_names[m] for m in top10],
                'score': (clf.coef_[n] * avgfreq)[top10]},
                index=range(1, 11))
else:
    # in case of a binary classification, negative weights are for the first class,
    # positive weights for the second
    top10 = numpy.argsort(clf.coef_[0] * avgfreq)[:10]
    topfeatures[clf.classes_[0]] = pandas.DataFrame({
            'word': [feature_names[m] for m in top10],
            'score': (clf.coef_[0] * avgfreq)[top10]},
            index=range(1, 11))
    top10 = numpy.argsort(clf.coef_[0] * avgfreq)[::-1][:10]
    topfeatures[clf.classes_[1]] = pandas.DataFrame({
            'word': [feature_names[m] for m in top10],
            'score': (clf.coef_[0] * avgfreq)[top10]},
            index=range(1, 11))
pandas.concat(topfeatures, axis=1)

In [ ]:
text1 ="# Sort the weights of the classifier and take top 10 itemstopfeatures = {}avgfreq = numpy.squeeze(numpy.asarray(X.mean(axis=0)))if len(clf.classes_) > 2:for n, target in enumerate(clf.classes_):top10 = numpy.argsort(clf.coef_[n] * avgfreq)[::-1][:10]topfeatures[target] = pandas.DataFrame({'word': [feature_names[m] for m in top10],'score': (clf.coef_[n] * avgfreq)[top10]},index=range(1, 11))else:# in case of a binary classification, negative weights are for the first class,# positive weights for the secondtop10 = numpy.argsort(clf.coef_[0] * avgfreq)[:10]topfeatures[clf.classes_[0]] = pandas.DataFrame({'word': [feature_names[m] for m in top10],'score': (clf.coef_[0] * avgfreq)[top10]},index=range(1, 11))top10 = numpy.argsort(clf.coef_[0] * avgfreq)[::-1][:10]topfeatures[clf.classes_[1]] = pandas.DataFrame({'word': [feature_names[m] for m in top10],'score': (clf.coef_[0] * avgfreq)[top10]},index=range(1, 11))pandas.concat(topfeatures, axis=1)"
text2 ="# Sort the weights of the classifier and take top 10 itemstopfeatures = {}avgfreq = numpy.squeeze(numpy.asarray(X.mean(axis=0)))if len(clf.classes_) > 2:for n, target in enumerate(clf.classes_):top10 = numpy.argsort(clf.coef_[n] * avgfreq)[::-1][:10]topfeatures[target] = pandas.DataFrame({'word': [feature_names[m] for m in top10],'score': (clf.coef_[n] * avgfreq)[top10]},index=range(1, 11))else:# in case of a binary classification, negative weights are for the first class,# positive weights for the secondtop10 = numpy.argsort(clf.coef_[0] * avgfreq)[:10]topfeatures[clf.classes_[0]] = pandas.DataFrame({'word': [feature_names[m] for m in top10],'score': (clf.coef_[0] * avgfreq)[top10]},index=range(1, 11))top10 = numpy.argsort(clf.coef_[0] * avgfreq)[::-1][:10]topfeatures[clf.classes_[1]] = pandas.DataFrame({'word': [feature_names[m] for m in top10],'score': (clf.coef_[0] * avgfreq)[top10]},index=range(1, 11))pandas.concat(topfeatures, axis=1)"

if text1 == text2: 
    print("hello")